# 第3章 分组

## 五、问题与练习

In [1]:
import numpy as np
import pandas as pd
df = pd.read_csv('data/table.csv',index_col='ID')
df.head()

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1102,S_1,C_1,F,street_2,192,73,32.5,B+
1103,S_1,C_1,M,street_2,186,82,87.2,B+
1104,S_1,C_1,F,street_2,167,81,80.4,B-
1105,S_1,C_1,F,street_4,159,64,84.8,B+


### 1. 问题

#### 【问题一】 什么是fillna的前向/后向填充，如何实现？

In [2]:
#产生数据
df_nan = df[['Math']].copy().reset_index().head()
df_nan.loc[2,['Math']]=np.nan
df_nan

,ID,Math
0,1101,34.0
1,1102,32.5
2,1103,NaN
3,1104,80.4
4,1105,84.8


In [3]:
#填充测试
print('ffile:\n',df_nan.fillna(method = 'ffill'))#前向填充，用NaN之前的最近非空值填充
print('bfile:\n',df_nan.fillna(method = 'bfill'))#后向填充，用NaN之后的最近非空值填充

ffile:
      ID  Math
0  1101  34.0
1  1102  32.5
2  1103  32.5
3  1104  80.4
4  1105  84.8
bfile:
      ID  Math
0  1101  34.0
1  1102  32.5
2  1103  80.4
3  1104  80.4
4  1105  84.8


#### 【问题二】 下面的代码实现了什么功能？请仿照设计一个它的groupby版本。
- 代码功能：对【0、1】Series列中连续非0位计数。

In [4]:
s = pd.Series ([0, 1, 1, 0, 1, 1, 1,0 ])
s1 = s.cumsum()#累加和
result = s.mul(s1).diff().where(lambda x: x < 0).ffill().add(s1,fill_value =0)

In [6]:
#groupby版本
for num in s.groupby(s).get_group(0).index:
    s1[num:] -= s1[num]
print(s1)

0    0
1    1
2    2
3    0
4    1
5    2
6    3
7    0
dtype: int64


#### 【问题三】 如何计算组内0.25分位数与0.75分位数？要求显示在同一张表上。


In [7]:
# 1、使用OrderedDict
from collections import OrderedDict
def f(df):
    data = OrderedDict()
    data['Q1'] = df.quantile(0.25)
    data['Q3'] = df.quantile(0.75)
    return pd.Series(data)
(df.groupby('School')['Math'].apply(f))

School    
S_1     Q1    41.850
        Q3    85.000
S_2     Q1    47.500
        Q3    72.225
Name: Math, dtype: float64

In [8]:
# 2、agg方法
def f(s,Q1,Q2):
    return s.quantile(Q1),s.quantile(Q2)
df.groupby('School')['Math'].agg(f,0.25,0.75)

School
S_1                (41.85, 85.0)
S_2    (47.5, 72.22500000000001)
Name: Math, dtype: object

#### 【问题四】 既然索引已经能够选出某些符合条件的子集，那么filter函数的设计有什么意义？
- filter具有筛选的整体性，用来筛选某些特定组。当且仅当该组所有数据都满足filter条件时整组保留。 
- 索引选出的子集是由满足条件的单条数据组成。

#### 【问题五】 整合、变换、过滤三者在输入输出和功能上有何异同？
- 聚合（Aggregation）标量，按组聚合成标量，支持自定义函数及lambda，可以利用NamedAgg进行多个函数聚合，带参数时使用wrap。
- 过滤（Filteration）矩阵，筛选某些符合条件组
- 变换（Transformation）向量，对组内的列进行统一处理，标量结果广播全组

#### 【问题六】 在带参数的多函数聚合时，有办法能够绕过wrap技巧实现同样功能吗？
- 可以，把需要聚合的函数f1\f2\f3...包装为一个函数f，参数通过f传入，return f1,f2,...，是多函数结果的元组

### 2. 练习
#### 【练习一】： 现有一份关于diamonds的数据集，列分别记录了克拉数、颜色、开采深度、价格，请解决下列问题：

In [9]:
df_d = pd.read_csv('data/Diamonds.csv')
df_d.head()

,carat,color,depth,price
0,0.23,E,61.5,326
1,0.21,E,59.8,326
2,0.23,E,56.9,327
3,0.29,I,62.4,334
4,0.31,J,63.3,335


#### (a) 在所有重量超过1克拉的钻石中，价格的极差是多少？
#### (b) 若以开采深度的0.2\0.4\0.6\0.8分位数为分组依据，每一组中钻石颜色最多的是哪一种？该种颜色是组内平均而言单位重量最贵的吗？
#### (c) 以重量分组(0-0.5,0.5-1,1-1.5,1.5-2,2+)，按递增的深度为索引排序，求每组中连续的严格递增价格序列长度的最大值。
#### (d) 请按颜色分组，分别计算价格关于克拉数的回归系数。（单变量的简单线性回归，并只使用Pandas和Numpy完成）

In [10]:
#(a) 在所有重量超过1克拉的钻石中，价格的极差是多少？
carat1 = df_d[df_d['carat']>1]
print('1 carat 以上 价格极差：',carat1['price'].max()-carat1['price'].min())

1 carat 以上 价格极差： 17561


In [11]:
#(b) 若以开采深度的0.2\0.4\0.6\0.8分位数为分组依据，
#    每一组中钻石颜色最多的是哪一种？该种颜色是组内平均而言单位重量最贵的吗？
def dq(x):#
    return df_d['depth'].quantile(x)
bins = [dq(0),dq(0.2),dq(0.4),dq(0.6),dq(0.8),dq(1)]
print(bins)
cuts = pd.cut(df_d['depth'],bins=bins) #可选label添加自定义标签
df_d['pc'] = df_d['price']/df_d['carat']#每颗单位价格
for name,group in df_d.groupby(cuts):
    
    colormost= group.groupby('color').count().sort_values(by = 'carat',ascending=False).index[0]
    print("颜色最多",colormost)
    
    mid = group.groupby('color').mean()
    #mid['pc'] = mid['price']/mid['carat']#总价格/总重量
    expercalat = mid['pc'].idxmax()
    print('单价最贵',expercalat,'\n')


[43.0, 60.8, 61.60000000000001, 62.10000000000001, 62.7, 79.0]
颜色最多 E
单价最贵 I 

颜色最多 G
单价最贵 F 

颜色最多 G
单价最贵 F 

颜色最多 G
单价最贵 G 

颜色最多 G
单价最贵 G 



In [12]:
#(c) 以重量分组(0-0.5,0.5-1,1-1.5,1.5-2,2+)，按递增的深度为索引排序，
#    求每组中连续的严格递增价格序列长度的最大值。
def f(s):
    mid = s.diff()
    length = 1
    maxlen = 1
    for i in range(0,len(mid)):
        if mid.iloc[i]>0:
            length += 1
            maxlen = max(maxlen,length)
        else:
            length = 1
    return maxlen

In [14]:
bins = [0,0.5,1,1.5,2,df_d['carat'].max()]
cuts = pd.cut(df_d['carat'],bins=bins)
for name,group in df_d.groupby(cuts):
    mid = (group.set_index('depth')['price'].sort_index())
    print(f(mid))

8
8
7
11
7


In [57]:
#(d) 请按颜色分组，分别计算价格关于克拉数的回归系数。
#   （单变量的简单线性回归，并只使用Pandas和Numpy.linalg完成）
import numpy as np
for name,group in df_d[['carat','price','color']].groupby('color'):
    X    = np.array([group['carat'],np.ones(group.shape[0])])
    Y    = np.array(group['price'])
    Beta = np.linalg.inv((X).dot(X.T)).dot(X).dot(Y)
    print(f'{name}色，回归系数：{Beta[0]:^12.6f}，截距：{Beta[1]:^12.6f}')

D色，回归系数：8408.353126 ，截距：-2361.017152
E色，回归系数：8296.212783 ，截距：-2381.049600
F色，回归系数：8676.658344 ，截距：-2665.806191
G色，回归系数：8525.345779 ，截距：-2575.527643
H色，回归系数：7619.098320 ，截距：-2460.418046
I色，回归系数：7761.041169 ，截距：-2878.150356
J色，回归系数：7094.192092 ，截距：-2920.603337


In [504]:
# 参考答案
for name,group in df_d[['carat','price','color']].groupby('color'):
    L1 = np.array([np.ones(group.shape[0]),group['carat']]).reshape(2,group.shape[0])
    L2 = group['price']
    result = (np.linalg.inv(L1.dot(L1.T)).dot(L1)).dot(L2).reshape(2,1)
    print('当颜色为%s时，截距项为：%f，回归系数为：%f'%(name,result[0],result[1]))

当颜色为D时，截距项为：-2361.017152，回归系数为：8408.353126
当颜色为E时，截距项为：-2381.049600，回归系数为：8296.212783
当颜色为F时，截距项为：-2665.806191，回归系数为：8676.658344
当颜色为G时，截距项为：-2575.527643，回归系数为：8525.345779
当颜色为H时，截距项为：-2460.418046，回归系数为：7619.098320
当颜色为I时，截距项为：-2878.150356，回归系数为：7761.041169
当颜色为J时，截距项为：-2920.603337，回归系数为：7094.192092


#### 【练习二】：有一份关于美国10年至17年的非法药物数据集，列分别记录了年份、州（5个）、县、药物类型、报告数量，请解决下列问题：

In [382]:
df_drug = pd.read_csv('data/Drugs.csv')
df_drug.head()

,YYYY,State,COUNTY,SubstanceName,DrugReports
0,2010,VA,ACCOMACK,Propoxyphene,1
1,2010,OH,ADAMS,Morphine,9
2,2010,PA,ADAMS,Methadone,2
3,2010,VA,ALEXANDRIA CITY,Heroin,5
4,2010,PA,ALLEGHENY,Hydromorphone,5


#### (a) 按照年份统计，哪个县的报告数量最多？这个县所属的州在当年也是报告数最多的吗？
#### (b) 从14年到15年，Heroin的数量增加最多的是哪一个州？它在这个州是所有药物中增幅最大的吗？若不是，请找出符合该条件的药物。

In [383]:
#(a) 按照年份统计，哪个县的报告数量最多？这个县所属的州在当年也是报告数最多的吗？
grouped_year = df_drug.groupby(['YYYY'])
for name,group in grouped_year:
    print(name)
    county = group.groupby('COUNTY')['DrugReports'].sum().sort_values(ascending=False).index[0]
    state  = group.groupby('State')['DrugReports'].sum().sort_values(ascending=False).index[0]
    
    if df_drug[(df_drug['COUNTY']==county)&(df_drug['State']==state)].shape[0] >0:
        thesame = '一致'
    else:
        thesame = '不一致'
    
    print('最多县',county,'最多州',state,thesame)

2010
最多县 PHILADELPHIA 最多州 PA 一致
2011
最多县 PHILADELPHIA 最多州 OH 不一致
2012
最多县 PHILADELPHIA 最多州 OH 不一致
2013
最多县 PHILADELPHIA 最多州 OH 不一致
2014
最多县 PHILADELPHIA 最多州 OH 不一致
2015
最多县 PHILADELPHIA 最多州 OH 不一致
2016
最多县 HAMILTON 最多州 OH 一致
2017
最多县 HAMILTON 最多州 OH 一致


In [384]:
#(b) 从14年到15年，Heroin的数量增加最多的是哪一个州？它在这个州是所有药物中增幅最大的吗？
#    若不是，请找出符合该条件的药物。
#先处理2014-2015heroin情况
df_14 = df_drug.loc[(df_drug['YYYY']==2014)&(df_drug['SubstanceName']=='Heroin')]
result14 = df_14.groupby(['State'])['DrugReports'].sum()

df_15 = df_drug.loc[(df_drug['YYYY']==2015)&(df_drug['SubstanceName']=='Heroin')]
result15 = df_15.groupby(['State'])['DrugReports'].sum()

state_heroin = (result15-result14).sort_values(ascending=False).index[0]
print('14-15Heroin报告增幅最大的州',state_heroin)


14-15Heroin报告增幅最大的州 OH


In [391]:
df_14_state = df_drug.loc[(df_drug['YYYY']==2014)&(df_drug['State']==state_heroin)]
result14_state = df_14_state.groupby('SubstanceName')['DrugReports'].sum()


df_15_state = df_drug.loc[(df_drug['YYYY']==2015)&(df_drug['State']==state_heroin)]
result15_state = df_15_state.groupby('SubstanceName')['DrugReports'].sum()

state_heroin_drug = (result15_state-result14_state).sort_values(ascending=False).index[0]

print(state_heroin,'州14-15报告增幅最大毒品',state_heroin_drug)

OH 州14-15报告增幅最大毒品 Heroin
